In [1]:
import json
import glob, pickle

import numpy as np
import pandas as pd

from qiskit import transpile
from qiskit import execute
from qiskit.providers.fake_provider import FakeLima
from qiskit.primitives import Estimator
from qiskit.circuit.random import random_circuit

import torch
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn.functional import dropout

from torch_geometric.nn import GCNConv, global_mean_pool, Linear, ChebConv, SAGEConv
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

from tqdm.notebook import tqdm_notebook
import matplotlib.pyplot as plt
import seaborn as sns

from blackwater.data.loaders.exp_val import CircuitGraphExpValMitigationDataset
from blackwater.data.generators.exp_val import exp_value_generator
from blackwater.data.utils import generate_random_pauli_sum_op
from blackwater.library.ngem.estimator import ngem

from qiskit.quantum_info import random_clifford

import random
from qiskit.circuit.library import HGate, SdgGate
from qiskit.circuit import ClassicalRegister

from blackwater.data.utils import (
    generate_random_pauli_sum_op,
    create_estimator_meas_data,
    circuit_to_graph_data_json,
    get_backend_properties_v1,
    encode_pauli_sum_op,
    create_meas_data_from_estimators
)
from blackwater.data.generators.exp_val import ExpValueEntry
from blackwater.metrics.improvement_factor import improvement_factor, Trial, Problem

from qiskit_aer import AerSimulator, QasmSimulator
from qiskit.providers.fake_provider import FakeMontreal, FakeLima, FakeBelem

from torch_geometric.nn import (
    GCNConv,
    TransformerConv,
    GATv2Conv,
    global_mean_pool,
    Linear,
    ChebConv,
    SAGEConv,
    ASAPooling,
    dense_diff_pool,
    avg_pool_neighbor_x
)
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.utils import to_dense_adj, to_dense_batch

from qiskit import QuantumCircuit
from qiskit.circuit.library import U3Gate, CZGate, PhaseGate, CXGate
from mbd_utils import construct_random_clifford, cal_z_exp, calc_imbalance, cal_all_z_exp, construct_mbl_circuit, generate_disorder
from gnn import ExpValCircuitGraphModel
plt.style.use({'figure.facecolor':'white'})

In [2]:
backend_lima = FakeLima()
backend_belem = FakeBelem()
backend_montreal = FakeMontreal()
properties_lima = get_backend_properties_v1(backend_lima)
properties_belem = get_backend_properties_v1(backend_belem)
properties_montreal = get_backend_properties_v1(backend_montreal)

## Local
backend_ideal = QasmSimulator() # Noiseless
backend_noisy_lima = AerSimulator.from_backend(backend_lima) # Noisy
backend_noisy_belem = AerSimulator.from_backend(backend_belem) # Noisy
backend_noisy_montreal = AerSimulator.from_backend(backend_montreal) # Noisy

run_config_ideal = {'shots': 10000, 'backend': backend_ideal, 'name': 'ideal'}
run_config_noisy_lima = {'shots': 10000, 'backend': backend_noisy_lima, 'name': 'noisy_lima'}
run_config_noisy_belem = {'shots': 10000, 'backend': backend_noisy_belem, 'name': 'noisy_belem'}
run_config_noisy_montreal = {'shots': 10000, 'backend': backend_noisy_montreal, 'name': 'noisy_montreal'}

BATCH_SIZE = 32

In [3]:
properties_lima.keys()

dict_keys(['name', 'gates_set', 'num_qubits', 'qubits_props', 'gate_props'])

In [4]:
properties_lima['gate_props']

{'id_0': {'index': 'id_0',
  'gate_error': 0.00019195510390342677,
  'gate_length': 35.55555555555556},
 'id_1': {'index': 'id_1',
  'gate_error': 0.00033064681663890665,
  'gate_length': 35.55555555555556},
 'id_2': {'index': 'id_2',
  'gate_error': 0.0001807726361160903,
  'gate_length': 35.55555555555556},
 'id_3': {'index': 'id_3',
  'gate_error': 0.0003925363414603665,
  'gate_length': 35.55555555555556},
 'id_4': {'index': 'id_4',
  'gate_error': 0.0006789531879417325,
  'gate_length': 35.55555555555556},
 'rz_0': {'index': 'rz_0', 'gate_error': 0, 'gate_length': 0},
 'rz_1': {'index': 'rz_1', 'gate_error': 0, 'gate_length': 0},
 'rz_2': {'index': 'rz_2', 'gate_error': 0, 'gate_length': 0},
 'rz_3': {'index': 'rz_3', 'gate_error': 0, 'gate_length': 0},
 'rz_4': {'index': 'rz_4', 'gate_error': 0, 'gate_length': 0},
 'sx_0': {'index': 'sx_0',
  'gate_error': 0.00019195510390342677,
  'gate_length': 35.55555555555556},
 'sx_1': {'index': 'sx_1',
  'gate_error': 0.0003306468166389066

In [27]:
def compare_backends(b1, b2, q1, q2):
    assert len(q1) == len(q2)
    diff_percent = []
    diff = []

    qubit_props_b1 = [b1['qubits_props'][i] for i in q1]
    qubit_props_b2 = [b2['qubits_props'][i] for i in q2]
    for key in ['readout_error']:
        for i in range(len(q1)):
            diff_percent += [abs(qubit_props_b1[i][key] - qubit_props_b2[i][key]) / qubit_props_b1[i][key]]
            diff += [abs(qubit_props_b1[i][key] - qubit_props_b2[i][key])]

    gates_b1 = [f'sx_{q}' for q in q1] + [f'x_{q}' for q in q1] + [f'cx_{i}_{j}' for i in q1 for j in q1 if i != j]
    gates_b2 = [f'sx_{q}' for q in q2] + [f'x_{q}' for q in q2] + [f'cx_{i}_{j}' for i in q2 for j in q2 if i != j]
    for key1, key2 in zip(gates_b1, gates_b2):
        try:
            diff_percent += [abs(b1['gate_props'][key1]['gate_error'] - b2['gate_props'][key2]['gate_error']) / b1['gate_props'][key1]['gate_error']]
            diff += [abs(b1['gate_props'][key1]['gate_error'] - b2['gate_props'][key2]['gate_error'])]
        except KeyError as e:
            print('No gate', e)

    return diff, diff_percent

q1 = q2 = list(range(5))
diff, diff_percent = compare_backends(properties_lima, properties_belem, q1, q2)
print(np.mean(diff), np.std(diff))
print(np.mean(diff_percent), np.std(diff_percent))

No gate 'cx_0_2'
No gate 'cx_0_3'
No gate 'cx_0_4'
No gate 'cx_1_4'
No gate 'cx_2_0'
No gate 'cx_2_3'
No gate 'cx_2_4'
No gate 'cx_3_0'
No gate 'cx_3_2'
No gate 'cx_4_0'
No gate 'cx_4_1'
No gate 'cx_4_2'
0.0059299205139710185 0.0082653308353876
0.5306288980423092 0.23227923520092497


In [25]:
def compare_backends(b1, b2, q1, q2):
    assert len(q1) == len(q2)
    diff_percent = []
    diff = []

    qubit_props_b1 = [b1['qubits_props'][i] for i in q1]
    qubit_props_b2 = [b2['qubits_props'][i] for i in q2]
    for key in ['t1', 't2']:
        for i in range(len(q1)):
            diff_percent += [abs(qubit_props_b1[i][key] - qubit_props_b2[i][key]) / qubit_props_b1[i][key]]
            diff += [abs(qubit_props_b1[i][key] - qubit_props_b2[i][key])]

    return diff, diff_percent

q1 = q2 = list(range(5))
diff, diff_percent = compare_backends(properties_lima, properties_montreal, q1, q2)
print(np.mean(diff), np.std(diff))
print(np.mean(diff_percent), np.std(diff_percent))

5.365487374858677e-05 5.009250950106034e-05
1.9933045214958696 3.247128977179777
